In [1]:
%load_ext autoreload
%autoreload 2

import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from datetime import datetime, timezone, tzinfo

from pyproj import Proj, CRS, Transformer

from sbp_tools.sbp_track_processing import process_track, save_track_to_radex, save_track, read_segypos

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

In [2]:
@dataclass
class GPRMC:
    time: str
    status: str
    lat: float
    lon: float
    sog: float
    tmg: float
    date: str
    mag_var: float
    mag_var_d: str
    datetime: datetime
    timestamp: float
    
@dataclass
class OpenCPN_track:
    lat: float
    lon: float
    datetime: datetime
    timestamp: float


def parse_nmea(nmea_file):
    nmea_rmc = []
    
    with open(nmea_file, 'r') as f1:
        fc = f1.read().splitlines()
        
        for line in fc:
            if line.startswith('$GPRMC'):
                lc = line.split(',')[1:]

                datetime_obj = datetime.strptime(lc[8]+lc[0][:-3], '%d%m%y%H%M%S').replace(tzinfo=timezone.utc)
                
                time=lc[0]
                status=lc[1]
                date=lc[8]
                timestamp=datetime_obj.timestamp() + int(lc[0][-2:])/100
                
                try:
                    lat=float(lc[2][0:2]) + float(lc[2][2:])/60 if lc[3]=='N' else float(lc[2][0:2]) + float(lc[2][2:])/60*-1
                    lon=float(lc[4][0:3]) + float(lc[4][3:])/60 if lc[5]=='E' else float(lc[4][0:3]) + float(lc[4][3:])/60*-1
                    sog=float(lc[6])
                    tmg=float(lc[7])
                    mag_var=float(lc[9])
                    mag_var_d=lc[10]
                    
                except:
                    lat=0
                    lon=0
                    sog=0
                    tmg=0
                    mag_var=0
                    mag_var_d='0'
                
                
                nmea_rmc.append(GPRMC(time,
                                      status,
                                      lat,
                                      lon,
                                      sog,
                                      tmg,
                                      date,
                                      mag_var,
                                      mag_var_d,
                                      datetime_obj,
                                      timestamp))
                
    return nmea_rmc
                

def parse_opencpn_track(opencpn_tr, head_time=4, head_lat=5, head_lon=6):
    
    open_cpn_tr = []
    
    with open(opencpn_tr, 'r') as f1:
        fc = f1.read().splitlines()
        
        for line in fc[1:]:
            lc = line.split(',')

            datetime_obj = datetime.strptime(lc[head_time],'%Y/%m/%d %H:%M:%S+00').replace(tzinfo=timezone.utc)
            
            lat=float(lc[head_lat])
            lon=float(lc[head_lon])
            timestamp=datetime_obj.timestamp()
            
            
            open_cpn_tr.append(OpenCPN_track(lat, lon, datetime_obj, timestamp))
    return open_cpn_tr
            
            

In [17]:
def take_every_10th(nmea_rmc_messages):
    filtered = []
    
    count = 1
    for msg in nmea_rmc_messages:
        if count == 1:
            filtered.append(msg)
            count += 1
        elif count == 10:
            count = 1
        else:
            count += 1
    
    return filtered

In [18]:
nmea_path = r'C:\YandexDisk\aa_cloudmbesproceessing\AI-68\NAV\Franson'
open_cpn = r'C:\Data\ai68_ais_track_KaporPoly.csv'

save_nmea_track_to =r'C:\YandexDisk\aa_cloudmbesproceessing\AI-68\NAV\NMEA_MERGED_every_10th.txt'

nmea_files = fes.file_ext_search('.nmea',nmea_path)

# nmea_rmc_messages = parse_nmea(nmea_path)

nmea_dict = {}

for nmea_file in nmea_files:
    filename = os.path.basename(nmea_file)
    nmea_rmc_messages = parse_nmea(nmea_file)
    
    
    
    nmea_dict[filename] = take_every_10th(nmea_rmc_messages)


with open(save_nmea_track_to, 'w') as f1:
    f1.write('nav_fname\ttimestamp\tISOdatetime\tlat\tlon\tyear\tyday\thour\tmin\tsec\tsog\ttmg\tstatus\n')
    
    for key in nmea_dict.keys():
        nmea_msg: GPRMC
        for nmea_msg in nmea_dict[key]:
            f1.write(f'{key}\t{nmea_msg.datetime.timestamp()}\t{nmea_msg.datetime.isoformat()}\t{nmea_msg.lat:.9f}\t{nmea_msg.lon:.9f}\t')
            f1.write(f'{nmea_msg.datetime.strftime('%Y\t%j\t%H\t%M\t%S')}\t{nmea_msg.sog:.3f}\t{nmea_msg.tmg:.3f}\t{nmea_msg.status}\n')
            
    
    
    

#open_cpn = parse_opencpn_track(open_cpn, head_time=4, head_lat=6, head_lon=5)
    

Searching *.nmea files in directory:C:\YandexDisk\aa_cloudmbesproceessing\AI-68\NAV\Franson


[GPRMC(time='111820.00', status='A', lat=59.90639960633333, lon=26.628666594, sog=3.975, tmg=25.934, date='090924', mag_var=10.5253, mag_var_d='E', datetime=datetime.datetime(2024, 9, 9, 11, 18, 20, tzinfo=datetime.timezone.utc), timestamp=1725880700.0),
 GPRMC(time='111821.00', status='A', lat=59.906416107666665, lon=26.628682268666665, sog=3.92, tmg=26.215, date='090924', mag_var=10.5253, mag_var_d='E', datetime=datetime.datetime(2024, 9, 9, 11, 18, 21, tzinfo=datetime.timezone.utc), timestamp=1725880701.0),
 GPRMC(time='111822.00', status='A', lat=59.90643219733333, lon=26.628696145333333, sog=4.102, tmg=24.794, date='090924', mag_var=10.5253, mag_var_d='E', datetime=datetime.datetime(2024, 9, 9, 11, 18, 22, tzinfo=datetime.timezone.utc), timestamp=1725880702.0),
 GPRMC(time='111823.00', status='A', lat=59.906449380666665, lon=26.628712614333335, sog=4.115, tmg=26.555, date='090924', mag_var=10.5253, mag_var_d='E', datetime=datetime.datetime(2024, 9, 9, 11, 18, 23, tzinfo=datetime.t

In [238]:
open_cpn

[OpenCPN_track(lat=59.979366667, lon=28.884876667, datetime=datetime.datetime(2024, 9, 11, 9, 52, 34, tzinfo=datetime.timezone.utc), timestamp=1726048354.0),
 OpenCPN_track(lat=59.979153333, lon=28.884893333, datetime=datetime.datetime(2024, 9, 11, 9, 52, 44, tzinfo=datetime.timezone.utc), timestamp=1726048364.0),
 OpenCPN_track(lat=59.97862, lon=28.884996667, datetime=datetime.datetime(2024, 9, 11, 9, 53, 11, tzinfo=datetime.timezone.utc), timestamp=1726048391.0),
 OpenCPN_track(lat=59.9782, lon=28.885196667, datetime=datetime.datetime(2024, 9, 11, 9, 53, 31, tzinfo=datetime.timezone.utc), timestamp=1726048411.0),
 OpenCPN_track(lat=59.977893333, lon=28.885436667, datetime=datetime.datetime(2024, 9, 11, 9, 53, 47, tzinfo=datetime.timezone.utc), timestamp=1726048427.0),
 OpenCPN_track(lat=59.977506667, lon=28.88588, datetime=datetime.datetime(2024, 9, 11, 9, 54, 10, tzinfo=datetime.timezone.utc), timestamp=1726048450.0),
 OpenCPN_track(lat=59.977273333, lon=28.886263333, datetime=datet

In [239]:
crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm35n, always_xy=True)


def make_array_ocpn(open_cpn_objs, transformer):
    
    arr_shape = (len(open_cpn_objs), 5)
    array = np.zeros(arr_shape)
    
    for num, obj in enumerate(open_cpn_objs):
        x, y = transformer.transform(obj.lon, obj.lat)
        
        array[num,0] = obj.timestamp
        array[num,1] = obj.lon
        array[num,2] = obj.lat
        array[num,3] = x
        array[num,4] = y
        
    return(array)
    
    
new_array = make_array_ocpn(open_cpn, transformer=transformer)
array_unique = np.unique(new_array, axis=0)

In [240]:
from scipy import interpolate

func_x = interpolate.interp1d(array_unique[:,0], array_unique[:,3])
func_y = interpolate.interp1d(array_unique[:,0], array_unique[:,4])



In [241]:
first = array_unique[0,0]
last = array_unique[-1,0]

print(first, last)

sec_times = np.arange(first, last+1,1)

print(sec_times)
print(len(sec_times))
print(last - first)

interp_x = func_x(sec_times)
interp_y = func_y(sec_times)

1726048354.0 1726062520.0
[1.72604835e+09 1.72604836e+09 1.72604836e+09 ... 1.72606252e+09
 1.72606252e+09 1.72606252e+09]
14167
14166.0


In [242]:
path = r'C:\Data\open_cpn_interpolated_1sec_SparkerKaporPoly_AISCOORDS.txt'

with open(path, 'w') as f2:
    f2.write('stmp,year,yyddmm,yday,hour,min,sec,x,y,datum\n')
    
    for num, timestamp in enumerate(sec_times):
        datetime_obj = datetime.fromtimestamp(timestamp, tz=timezone.utc)
        datum = 'wgs84utm35n'
        
        f2.write(f'{timestamp},{datetime_obj.strftime('%Y,%y%m%d,%j,%H,%M,%S')},{interp_x[num]:.5f},{interp_y[num]:.5f},{datum}\n')
        
        
        
        
        
def proc_track_from_pds(paths, save_to, cruisename, delimiter=','):
    delimeter = delimiter

    yearday_dict = {}

    for trackfile in paths:
        with open(trackfile, 'r') as f1:
            file_content = f1.read().splitlines()
            
            for line in file_content:
                line_content = line.split(delimeter)
                
                year_day = line_content[3]
                        
                if year_day in yearday_dict.keys():
                    pass
                else:
                    yearday_dict[year_day] = []
                
                file_string = line
                yearday_dict[year_day].append(file_string)
                
    for key_yearday in yearday_dict.keys():
        trackfile_yearday = os.path.join(save_to, cruisename + '_' + key_yearday + '.txt')
        
        with open(trackfile_yearday, 'w') as f2:
            f2.write('stmp,year,yyddmm,yday,hour,min,sec,x,y,datum\n')
            
            yearday_dict[key_yearday].sort()
            for line in yearday_dict[key_yearday]:
                f2.write(f'{line}\n')

In [234]:
def proc_track_from_pds(paths, save_to, cruisename, delimiter=','):
    delimeter = delimiter

    yearday_dict = {}

    for trackfile in paths:
        with open(trackfile, 'r') as f1:
            file_content = f1.read().splitlines()
            
            for line in file_content[1:]:
                line_content = line.split(delimeter)
                
                year_day = line_content[3]
                        
                if year_day in yearday_dict.keys():
                    pass
                else:
                    yearday_dict[year_day] = []
                
                file_string = line
                yearday_dict[year_day].append(file_string)
                
    for key_yearday in yearday_dict.keys():
        trackfile_yearday = os.path.join(save_to, cruisename + '_' + key_yearday + '.txt')
        
        with open(trackfile_yearday, 'w') as f2:
            f2.write('stmp,year,yyddmm,yday,hour,min,sec,x,y,datum\n')
            
            yearday_dict[key_yearday].sort()
            for line in yearday_dict[key_yearday]:
                f2.write(f'{line}\n')

In [243]:
path = r'C:\Data\open_cpn_interpolated_1sec_SparkerKaporPoly_AISCOORDS.txt'
sort_to = r'C:\Data\sorted_by_yearday'

proc_track_from_pds([path], sort_to, 'AI68_KAPOR')

In [29]:
date = '300824'
time = '000358.00'

dtime = datetime.strptime(date+time, '%d%m%y%H%M%S.00').replace(tzinfo=timezone.utc)

print(dtime)
print(dtime.timestamp())
tmst = dtime.timestamp()

tt = datetime.fromtimestamp(tmst, tz=timezone.utc)
print(tt)

2024-08-30 00:03:58+00:00
1724976238.0
2024-08-30 00:03:58+00:00


In [31]:
date = '2024/09/06 09:13:13+00'

dtime = datetime.strptime(date,'%Y/%m/%d %H:%M:%S+00')

print(dtime)
print(dtime.timestamp())
tmst = dtime.timestamp()

tt = datetime.fromtimestamp(tmst)
print(tt)

2024-09-06 09:13:13
1725606793.0
2024-09-06 09:13:13


In [ ]:
sort_to = r'C:\Data\sorted_by_yearday'


def proc_track_from_pds(trackfile_paths, save_to, cruisename, delimiter=','):
    delimeter = delimiter

    yearday_dict = {}

    for trackfile in trackfile_paths:
        with open(trackfile, 'r') as f1:
            file_content = f1.read().splitlines()
            
            for line in file_content:
                line_content = line.split(delimeter)
                
                posix_time = float(line_content[0])
                datetime_pds = line_content[1]
                x_coord = float(line_content[2])
                y_coord = float(line_content[3])
                
                datetime_conv = datetime.fromtimestamp(posix_time, tz=timezone.utc)
                
                year_day = datetime_conv.strftime('%Y_%j')
                        
                if year_day in yearday_dict.keys():
                    pass
                else:
                    yearday_dict[year_day] = []
                
                file_string = f'{posix_time:.05f} {x_coord:.04f} {y_coord:.04f} '+ datetime_conv.strftime('%Y %m %d %j %H %M %S %f')
                yearday_dict[year_day].append(file_string)
                
    for key_yearday in yearday_dict.keys():
        trackfile_yearday = os.path.join(save_to, cruisename + '_' + key_yearday + '.txt')
        
        with open(trackfile_yearday, 'w') as f2:
            f2.write('posix_timestamp x y year day month yearday hour minute second microsecond\n')
            
            yearday_dict[key_yearday].sort()
            for line in yearday_dict[key_yearday]:
                f2.write(f'{line}\n')